# Description

This notebook is meant to test functionalities of libigl out. Code is mostly taken from: https://geometryprocessing.github.io/blackbox-computing-python/geo_viz/.

# Load libraries

It seems like igl cannot be imported from jupyter lab for some reason. It seems to work fine with jupyter notebook and plain python files.

In [1]:
import igl
import scipy as sp
import numpy as np
import meshplot as mp

# Plot two triangles

In [2]:
V = np.array([
    [0., 0, 0],
    [1, 0, 0],
    [1, 1, 1],
    [2, 1, 0]
])

F = np.array([
    [0, 1, 2],
    [1, 3, 2]
])

mp.plot(V, F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.0, 0.5,…

# Load data

In [3]:
# Read oloid and retrieve shape
v, f = igl.read_triangle_mesh("meshes/oloid64_tri.obj")
print("===== Characteristics of the mesh =====")
print("Number of vertices: {}".format(v.shape[0]))
print("Number of faces: {}".format(f.shape[0]))

# Write oloid
igl.write_triangle_mesh("meshes/oloid64_written.off", v, f)

# Visualize oloid
mp.plot(v, f)

===== Characteristics of the mesh =====
Number of vertices: 66
Number of faces: 128


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.5,…

In [4]:
# Read pig and retrieve shape
v, f = igl.read_triangle_mesh("meshes/pig.obj")
print("===== Characteristics of the mesh =====")
print("Number of vertices: {}".format(v.shape[0]))
print("Number of faces: {}".format(f.shape[0]))

# Visualize oloid
mp.plot(v, f)

===== Characteristics of the mesh =====
Number of vertices: 5744
Number of faces: 11392


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

# Compute normals

## Face normals

In [5]:
nf = igl.per_face_normals(v, f, np.zeros(3))
print("Shape of the face normals: {}".format(nf.shape))
mp.plot(v, f, c=np.abs(nf), shading={"roughness": 1.0})

Shape of the face normals: (11392, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

## Vertex normal

In [6]:
nv = igl.per_vertex_normals(v, f)
print("Shape of the vertex normals: {}".format(nv.shape))

p = mp.plot(v, f, c=np.abs(nv), shading={"roughness": 1.0})
p.add_lines(v, v + nv * 2e-2)

Shape of the vertex normals: (5744, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

1

# Curvature

## Compute curvature

In [7]:
# Calculate principal curvature
d1, d2, k1, k2 = igl.principal_curvature(v, f)
print("First principal directions shape: {}".format(d1.shape))
print("First principal curvature shape: {}".format(k1.shape))

# Calculate mean curvature for color coding
mean_curv = 0.5 * (k1 + k2)

p = mp.plot(v, f, c=mean_curv)

p.add_lines(v + d1 * 1e-2, v - d1 * 1e-2, shading={"line_color": "red"})
p.add_lines(v + d2 * 1e-2, v - d2 * 1e-2, shading={"line_color": "yellow"})

First principal directions shape: (5744, 3)
First principal curvature shape: (5744,)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

2

## Save results

In [8]:
p = mp.plot(v, f, c=np.random.rand(*f.shape), filename="output/testPig.html")
p.save("output/testPig.html")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

Plot saved to file output/testPig.html.


# Mesh Statistics

In [9]:
# Irregular vertices, the border is ignored
irregular = igl.is_irregular_vertex(v, f) 
irregular_ratio = np.sum(irregular) / v.shape[0]
print("Irregular vertices:\n%d/%d (%.2f%%)\n"%
      (np.sum(irregular), v.shape[0], irregular_ratio * 100))

# Compute areas, min, max and std
area = igl.doublearea(v, f) / 2.0
area_avg = np.mean(area)
area_min = np.min(area) / area_avg
area_max = np.max(area) / area_avg
area_ns = (area - area_avg) / area_avg
area_sigma = np.sqrt(np.mean(np.square(area_ns)))
print("Areas Min/Max/Sigma: \n%.2f/%.2f/%.2f\n"%
      (area_min, area_max, area_sigma))

# Compute per face angles, min, max and std
angles = igl.internal_angles(v, f)
angles = 360.0 * (angles / (2 * np.pi))
angle_avg = np.mean(angles)
angle_min = np.min(angles)
angle_max = np.max(angles)
angle_ns = angles - angle_avg
angle_sigma = np.sqrt(np.mean(np.square(angle_ns)))

print("Angles in degrees Min/Max/Avg/Sigma: \n%.2f/%.2f/%.2f/%.2f\n"%
      (angle_min, angle_max, angle_avg, angle_sigma))

print("Some array shapes:")
print("Areas shape: {}".format(area.shape))
print("Angles shape: {}".format(angles.shape))


Irregular vertices:
5660/5744 (98.54%)

Areas Min/Max/Sigma: 
0.00/11.74/1.51

Angles in degrees Min/Max/Avg/Sigma: 
2.07/173.09/60.00/29.67

Some array shapes:
Areas shape: (11392,)
Angles shape: (11392, 3)


# Texture Mapping

# Laplacian Smoothing

In [18]:
from scipy.sparse.linalg import spsolve
import time
from IPython import display

v, f = igl.read_triangle_mesh("meshes/pig.obj")

l = igl.cotmatrix(v, f)
vs = [v]
for i in range(10):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    # Diffusion step (M - dt.L).V_{t+1} = M.V_{t}
    v = spsolve(m - 0.001 * l, m.dot(v))
    vs.append(v)

for vert in vs:
    display.clear_output(wait=True)
    mp.plot(vert, f)
    time.sleep(0.5)
    
print("Shape of the Cotangent Laplacian array: {}".format(l.shape))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.102102…

Shape of the Cotangent Laplacian array: (5744, 5744)


# Shape Filtering

## Spectral Filtering

In [24]:
from scipy.sparse.linalg import eigsh

v, f = igl.read_triangle_mesh("meshes/pig.obj")
l = igl.cotmatrix(v, f)

# Keep the smallest 200 eigenvalues (and their corresponding eigenvectors)
d, u = eigsh(-l, 200, which="SM")
vs = u @ u.T @ v

# Project vertices positions on the eigenvectors associated with the smallest eigenvalues
mp.plot((u[:, :30] @ u[:, :30].T) @ v, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.011995…

## Visualize Eigenvectors

In [31]:
n_eigvec = 6
v, f = igl.read_triangle_mesh("meshes/pig.obj")
mp.plot(v, f, c=u[:, n_eigvec])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

# Geodesics

In [32]:
v, f = igl.read_triangle_mesh("meshes/pig.obj")

# Select a vertex as origin
vs = np.array([0])

# All vertices are the targets
vt = np.arange(v.shape[0])

d = igl.exact_geodesic(v, f, vs, vt)

# Visualize with periodic function
c = np.abs(np.sin((d / 0.03 * np.pi)))
p = mp.plot(v, f, c)
p.add_points(v[vs], shading={"point_size": 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0067104…

1